In [1]:
import os
import numpy as np
import pandas as pd

from tqdm import tqdm

import wavio
from scipy.io import wavfile
from librosa.core import resample, to_mono
from glob import glob


In [8]:
src_root = 'raw_data'
dst_root = 'clean'
dt = 1.0
sample_f = 44100
dummy_file = 'finger_snaps_1_4'
threshold = 120

In [9]:
def check_dir(path):
    if os.path.exists(path) is False:
        os.mkdir(path)

In [10]:
def envelope(y, rate, t):
    mask = []
    y = pd.Series(y).apply(np.abs)
    y_mean = y.rolling(window = int(rate/30), min_periods=1,center=True).mean()
    for mean in y_mean:
        if mean > threshold:
            mask.append(True)
        else:
            mask.append(False)
    return mask, y_mean

In [11]:
def downsample_mono(path, sf):
    obj = wavio.read(path)
    wav = obj.data.astype(np.float32,order='F')
    rate = obj.rate
    try:
        channel = wav.shape[1]
        if channel == 2:
            wav = to_mono(wav.T)
        elif channel == 1:
            wav = to_mono(wav.reshape(-1))
    except IndexError:
        wav = to_mono(wav.reshape(-1))
        pass
    except Exception as exc:
        raise exc
    wav = resample(wav, orig_sr = rate, target_sr = sf)
    wav = wav.astype(np.int16)
    return sf, wav

In [12]:
def save_sample(sample, rate, target_dir, f, ix):
    f = f.split('.wav')[0]
    dst_path = os.path.join(target_dir.split('.')[0],f + f"{ix}.wav")
    if os.path.exists(dst_path):
        return
    wavfile.write(dst_path,rate,sample)

In [14]:
def split_wavs(src = src_root, dst = dst_root, dt = dt,sf = sample_f,t = threshold):
    wav_paths = glob('{}/**'.format(src), recursive=True)
    wav_paths = [x for x in wav_paths if '.wav' in x]
    check_dir(dst)
    classes = os.listdir(src)
    for _cls in classes:
        target_dir = os.path.join(dst,_cls)
        check_dir(target_dir)
        from_dir = os.path.join(src,_cls)
        for f in tqdm(os.listdir(from_dir)):
            src_f = os.path.join(from_dir,f)
            rate,wav = downsample_mono(src_f,sf)
            
            delta_sample = int(dt*rate)
            trunc = wav.shape[0] % delta_sample
            for cnt, i in enumerate(np.arange(0,wav.shape[0]-trunc,delta_sample)):
                start = int(i)
                stop = int(i+delta_sample)
                raw_sample = wav[start:stop]
                mask, env = envelope(raw_sample, rate,t)
                noise_free_sample = raw_sample[mask]
                sample = np.zeros(shape = (delta_sample,),dtype = np.int16)
                sample[:noise_free_sample.shape[0]] = noise_free_sample
                save_sample(sample,rate,target_dir,f,cnt)


In [15]:
split_wavs()

100%|██████████| 4/4 [00:36<00:00,  9.05s/it]
